In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.ticker as mtick
import arcpy

# Import the required ArcGIS API for Python modules
import arcgis
gis = arcgis.GIS()

from IPython.display import display, Markdown

In [2]:
#get root folder
dir_working = os.getcwd()
print(dir_working)

#define folders
dir_data     = os.path.join(dir_working, "data"        )
dir_process  = os.path.join(dir_working, "intermediate")
dir_results  = os.path.join(dir_working, "results"     )
dir_inputs   = os.path.join(dir_working, "input"       )

ClassParameters    = os.path.join(dir_inputs, r"class_parameters.csv"     )
CountiesTable      = os.path.join(dir_inputs ,r"counties.csv"             )

ProcessGDB = "process.gdb"

#name of new data features
AnalysisAreas      = os.path.join(dir_process, "analysisareas.shp"        )
AnalysisAreas_IDs  = os.path.join(dir_process, "analysisareas_ids.shp"    )
ParcelsAA          = os.path.join(dir_process, "parcelsaa.shp"            )

AnalysisAreasTable = os.path.join(dir_results, "areas.csv"                )



def deleteIfExists(obj):
    if arcpy.Exists(obj): arcpy.Delete_management(obj)


E:\GitHub\Residential-Capacity


In [3]:
AnalysisAreas

'E:\\GitHub\\Residential-Capacity\\intermediate\\analysisareas.shp'

In [4]:
sdf_AA = pd.DataFrame.spatial.from_featureclass(AnalysisAreas)
#sdf_AA

sdf_AAid = pd.DataFrame.spatial.from_featureclass(AnalysisAreas_IDs)
sdf_AAid

,FID,Shape_Leng,Shape_Area,AreaID,ClassID,SHAPE
0,0,599.400903,1.220382e+04,4,a4,"{""rings"": [[[420271.3705000002, 4480517.338500..."
1,1,1554.841774,8.561669e+04,4,a4,"{""rings"": [[[420636.9475999996, 4481132.772299..."
2,2,254.967270,3.811046e+03,4,a4,"{""rings"": [[[420671.3481999999, 4481261.932600..."
3,3,2666.392431,2.609549e+05,4,a4,"{""rings"": [[[420869.50710000005, 4482955.5603]..."
4,4,11708.780939,1.126298e+06,4,a4,"{""rings"": [[[421226.04870000016, 4479294.0206]..."
...,...,...,...,...,...,...
10971,10971,2229.618596,2.944859e+05,10,g3,"{""rings"": [[[414942.4210000001, 4550112.225299..."
10972,10972,856.947230,2.329713e+04,10,g3,"{""rings"": [[[417057.5292999996, 4550162.428400..."
10973,10973,966.598419,5.007495e+04,10,g3,"{""rings"": [[[416772.02419999987, 4550546.96120..."
10974,10974,8681.643024,8.610622e+05,10,g3,"{""rings"": [[[414009.90950000007, 4551759.66699..."


In [5]:
#create map centered on Salt Lake
map_areas = gis.map('Salt Lake')

#expression to classify enrollment
arcade_expression_aa = ("return 'classAA';")

#symbology for enrollment classes
uv_aa = [{"value":"classAA", "label":"Analysis Area", "symbol":{"type":"esriSFS","color":[128,128,128,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}}]

#classify land use
arcade_expression_aa = ("var v = $feature.Analysis_1;"
                        "if      (v=='NONTOD|Metropolitan Center') { return 'mc'    ; }"
                        "else if (v=='CRT|Metropolitan Center'   ) { return 'mc_tod'; }"
                        "else if (v=='LRT|Metropolitan Center'   ) { return 'mc_tod'; }"
                        "else if (v=='NONTOD|Urban Center'       ) { return 'uc'    ; }"
                        "else if (v=='CRT|Urban Center'          ) { return 'uc_tod'; }"
                        "else if (v=='LRT|Urban Center'          ) { return 'uc_tod'; }"
                        "else if (v=='NONTOD|City Center'        ) { return 'cc'    ; }"
                        "else if (v=='CRT|City Center'           ) { return 'cc_tod'; }"
                        "else if (v=='LRT|City Center'           ) { return 'cc_tod'; }"
                        "else if (v=='CRT|NA'                    ) { return 'nc_tod'; }"
                        "else if (v=='LRT|NA'                    ) { return 'nc_tod'; }"
                        "else                                      { return 'other' ; }")

#symbology for enrollment classes
uv_aa = [  
          {"value":"mc"    , "label":"Metropolitan Center"         , "symbol":{"type":"esriSFS","color":[128,  0,128,100], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
          {"value":"mc_tod", "label":"Metropolitan Center with TOD", "symbol":{"type":"esriSFS","color":[128,  0,128,200], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
          {"value":"uc"    , "label":"Urban Center"                , "symbol":{"type":"esriSFS","color":[  0,  0,128,100], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
          {"value":"uc_tod", "label":"Urban Center with TOD"       , "symbol":{"type":"esriSFS","color":[  0,  0,128,200], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
          {"value":"cc"    , "label":"City Center"                 , "symbol":{"type":"esriSFS","color":[255,  0,  0,100], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
          {"value":"cc_tod", "label":"City Center with TOD"        , "symbol":{"type":"esriSFS","color":[255,  0,  0,200], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
          {"value":"nc_tod", "label":"TOD Only"                    , "symbol":{"type":"esriSFS","color":[  0,255,  0,200], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
          {"value":"other" , "label":"Other"                       , "symbol":{"type":"esriSFS","color":[128,128,128,200], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}}
        ]

#define sdf layer
sdf_AA.spatial.plot(map_widget = map_areas,
                                 renderer_type='u-a', #'u-a' stands for uniqe value with arcade expression
                                 unique_values=uv_aa,
                                 arcade_expression=arcade_expression_aa,
                                 default_symbol="" #don't include an 'other' category
                                 )

#define map characteristics
map_areas.layout.height='500px'
map_areas.legend=True

#map title
display(Markdown('<h2><center>Analysis Areas</center></h2>'))

#display map
map_areas

<h2><center>Analysis Areas</center></h2>

MapView(layout=Layout(height='500px', width='100%'), legend=True)

# Parameters

In [6]:
df_ClassParam = pd.read_csv(ClassParameters)
display(df_ClassParam)

,ClassID,ClassDescription,SFSplitRes,SFSplitCom,CapacityRes_DUA,CapacityCom_FAR,SFperHH,SFperEmp,PercentOpenSpace,RedevValuePerAcreRes,RedevValuePerAcreCom,RedevAndOr,RedevBldgAgeRes_Low,RedevBldgAgeRes_High,RedevBldgAgeCom_Low,RedevBldgAgeCom_High,RedevProb,ClassOrder
0,f4,Multifamily Metro,0.40,0.60,120.0,15.0,1500,225,0.10,5000000,4000000,AND,60,120,20,120,1,29
1,f3,Multifamily City/TOD,0.50,0.50,60.0,4.0,1500,250,0.10,3000000,2400000,AND,60,120,20,120,1,28
2,f2,Multifamily Suburban,0.70,0.30,30.0,1.0,1500,300,0.10,1500000,1200000,AND,60,120,20,120,1,27
3,f1,Multifamily Single Story,0.85,0.15,10.0,0.5,2000,300,0.10,1000000,800000,AND,60,300,20,300,1,26
4,g4,Mixed-Use Metro,0.40,0.60,120.0,15.0,1500,225,0.10,5000000,4000000,AND,60,120,20,120,1,33
5,g3,Mixed-Use City/TOD,0.50,0.50,60.0,4.0,1500,250,0.10,3000000,2400000,AND,60,120,20,120,1,32
6,g2,Mixed-Use Suburban,0.70,0.30,30.0,1.0,1500,300,0.10,1500000,1200000,AND,60,120,20,120,1,31
7,g1,Mixed-Use Single Story,0.85,0.15,10.0,0.5,2000,300,0.10,1000000,800000,AND,60,300,20,300,1,30
8,h4,Mixed Residential Metro,0.40,0.60,120.0,15.0,1500,225,0.10,5000000,4000000,AND,60,120,20,120,1,37
9,h3,Mixed Residential City/TOD,0.50,0.50,60.0,4.0,1500,250,0.10,3000000,2400000,AND,60,120,20,120,1,36


In [7]:
df_ClassParam.dtypes

ClassID                  object
ClassDescription         object
SFSplitRes              float64
SFSplitCom              float64
CapacityRes_DUA         float64
CapacityCom_FAR         float64
SFperHH                   int64
SFperEmp                  int64
PercentOpenSpace        float64
RedevValuePerAcreRes      int64
RedevValuePerAcreCom      int64
RedevAndOr               object
RedevBldgAgeRes_Low       int64
RedevBldgAgeRes_High      int64
RedevBldgAgeCom_Low       int64
RedevBldgAgeCom_High      int64
RedevProb                 int64
ClassOrder                int64
dtype: object

In [8]:
df_AADevCat = pd.read_csv(os.path.join(dir_results, r'AnalysisAreas_byDevCategory.csv'))
df_AADevCat

,AreaID,ClassID,county_id,DevCategory,parcel_id,job_spaces,resunits,Acres,sf_res,sf_com,acres_res,acres_com
0,0,a1,0.0,Developable,18.0,0.000000,0.000000,15.567788,0.000000,0.000000,0.000000,0.000000
1,0,a1,0.0,Undevelopable,8.0,0.000000,0.000000,14.733847,0.000000,0.000000,0.000000,0.000000
2,0,a1,1.0,Undevelopable,1.0,0.000000,0.000000,0.108287,0.000000,0.000000,0.000000,0.000000
3,0,a1,2.0,Developable,102.0,0.000000,0.000000,234.281314,0.000000,0.000000,0.000000,0.000000
4,0,a1,2.0,Redevelopable,9.0,1.470310,9.117647,84.854662,13154.026381,1433.552632,83.921666,0.932996
...,...,...,...,...,...,...,...,...,...,...,...,...
2357,77,f2,1.0,Undevelopable,550.0,13.989540,0.000000,15.171574,0.000000,5232.241785,0.000000,0.000000
2358,77,g2,1.0,Developable,39.0,0.000000,0.000000,94.609768,0.000000,0.000000,0.000000,0.000000
2359,77,g2,1.0,Redevelopable,2.0,0.000109,0.999174,0.320516,1187.019228,0.040774,0.320457,0.000059
2360,77,g2,1.0,Remain,14.0,0.002325,6.064735,5.687062,15878.834261,1.471514,5.682825,0.004237


# Calculate New Jobs/HH

In [9]:
#join Analysis Areas with class parameters
df_AADevCatwParam = pd.DataFrame.merge(df_AADevCat,df_ClassParam,on='ClassID',how='left')
df_AADevCatwParam

,AreaID,ClassID,county_id,DevCategory,parcel_id,job_spaces,resunits,Acres,sf_res,sf_com,...,PercentOpenSpace,RedevValuePerAcreRes,RedevValuePerAcreCom,RedevAndOr,RedevBldgAgeRes_Low,RedevBldgAgeRes_High,RedevBldgAgeCom_Low,RedevBldgAgeCom_High,RedevProb,ClassOrder
0,0,a1,0.0,Developable,18.0,0.000000,0.000000,15.567788,0.000000,0.000000,...,0.15,40000,1000000,AND,50,300,20,300,1,1
1,0,a1,0.0,Undevelopable,8.0,0.000000,0.000000,14.733847,0.000000,0.000000,...,0.15,40000,1000000,AND,50,300,20,300,1,1
2,0,a1,1.0,Undevelopable,1.0,0.000000,0.000000,0.108287,0.000000,0.000000,...,0.15,40000,1000000,AND,50,300,20,300,1,1
3,0,a1,2.0,Developable,102.0,0.000000,0.000000,234.281314,0.000000,0.000000,...,0.15,40000,1000000,AND,50,300,20,300,1,1
4,0,a1,2.0,Redevelopable,9.0,1.470310,9.117647,84.854662,13154.026381,1433.552632,...,0.15,40000,1000000,AND,50,300,20,300,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2357,77,f2,1.0,Undevelopable,550.0,13.989540,0.000000,15.171574,0.000000,5232.241785,...,0.10,1500000,1200000,AND,60,120,20,120,1,27
2358,77,g2,1.0,Developable,39.0,0.000000,0.000000,94.609768,0.000000,0.000000,...,0.10,1500000,1200000,AND,60,120,20,120,1,31
2359,77,g2,1.0,Redevelopable,2.0,0.000109,0.999174,0.320516,1187.019228,0.040774,...,0.10,1500000,1200000,AND,60,120,20,120,1,31
2360,77,g2,1.0,Remain,14.0,0.002325,6.064735,5.687062,15878.834261,1.471514,...,0.10,1500000,1200000,AND,60,120,20,120,1,31


In [10]:
#remove percent open space from available acreage
df_AADevCatwParam['AcresOpenSpaceRemoved'] = df_AADevCatwParam['Acres'] * (1-df_AADevCatwParam['PercentOpenSpace'])
pd.set_option('display.max_columns', None)
display(df_AADevCatwParam)
display(df_AADevCatwParam.columns)

,AreaID,ClassID,county_id,DevCategory,parcel_id,job_spaces,resunits,Acres,sf_res,sf_com,acres_res,acres_com,ClassDescription,SFSplitRes,SFSplitCom,CapacityRes_DUA,CapacityCom_FAR,SFperHH,SFperEmp,PercentOpenSpace,RedevValuePerAcreRes,RedevValuePerAcreCom,RedevAndOr,RedevBldgAgeRes_Low,RedevBldgAgeRes_High,RedevBldgAgeCom_Low,RedevBldgAgeCom_High,RedevProb,ClassOrder,AcresOpenSpaceRemoved
0,0,a1,0.0,Developable,18.0,0.000000,0.000000,15.567788,0.000000,0.000000,0.000000,0.000000,Single Family A1,1.0,0.0,1.5,0.5,4000,350,0.15,40000,1000000,AND,50,300,20,300,1,1,13.232620
1,0,a1,0.0,Undevelopable,8.0,0.000000,0.000000,14.733847,0.000000,0.000000,0.000000,0.000000,Single Family A1,1.0,0.0,1.5,0.5,4000,350,0.15,40000,1000000,AND,50,300,20,300,1,1,12.523770
2,0,a1,1.0,Undevelopable,1.0,0.000000,0.000000,0.108287,0.000000,0.000000,0.000000,0.000000,Single Family A1,1.0,0.0,1.5,0.5,4000,350,0.15,40000,1000000,AND,50,300,20,300,1,1,0.092044
3,0,a1,2.0,Developable,102.0,0.000000,0.000000,234.281314,0.000000,0.000000,0.000000,0.000000,Single Family A1,1.0,0.0,1.5,0.5,4000,350,0.15,40000,1000000,AND,50,300,20,300,1,1,199.139117
4,0,a1,2.0,Redevelopable,9.0,1.470310,9.117647,84.854662,13154.026381,1433.552632,83.921666,0.932996,Single Family A1,1.0,0.0,1.5,0.5,4000,350,0.15,40000,1000000,AND,50,300,20,300,1,1,72.126463
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2357,77,f2,1.0,Undevelopable,550.0,13.989540,0.000000,15.171574,0.000000,5232.241785,0.000000,0.000000,Multifamily Suburban,0.7,0.3,30.0,1.0,1500,300,0.10,1500000,1200000,AND,60,120,20,120,1,27,13.654417
2358,77,g2,1.0,Developable,39.0,0.000000,0.000000,94.609768,0.000000,0.000000,0.000000,0.000000,Mixed-Use Suburban,0.7,0.3,30.0,1.0,1500,300,0.10,1500000,1200000,AND,60,120,20,120,1,31,85.148791
2359,77,g2,1.0,Redevelopable,2.0,0.000109,0.999174,0.320516,1187.019228,0.040774,0.320457,0.000059,Mixed-Use Suburban,0.7,0.3,30.0,1.0,1500,300,0.10,1500000,1200000,AND,60,120,20,120,1,31,0.288464
2360,77,g2,1.0,Remain,14.0,0.002325,6.064735,5.687062,15878.834261,1.471514,5.682825,0.004237,Mixed-Use Suburban,0.7,0.3,30.0,1.0,1500,300,0.10,1500000,1200000,AND,60,120,20,120,1,31,5.118356


Index(['AreaID', 'ClassID', 'county_id', 'DevCategory', 'parcel_id',
       'job_spaces', 'resunits', 'Acres', 'sf_res', 'sf_com', 'acres_res',
       'acres_com', 'ClassDescription', 'SFSplitRes', 'SFSplitCom',
       'CapacityRes_DUA', 'CapacityCom_FAR', 'SFperHH', 'SFperEmp',
       'PercentOpenSpace', 'RedevValuePerAcreRes', 'RedevValuePerAcreCom',
       'RedevAndOr', 'RedevBldgAgeRes_Low', 'RedevBldgAgeRes_High',
       'RedevBldgAgeCom_Low', 'RedevBldgAgeCom_High', 'RedevProb',
       'ClassOrder', 'AcresOpenSpaceRemoved'],
      dtype='object')

In [18]:
#calculate Com and Res Acreage

#initialize - acres_remain and acres_developed don't have res/com breakdown
df_AADevCatwParam['acres_undevelopable'] = 0.0
df_AADevCatwParam['acres_remain'       ] = 0.0
df_AADevCatwParam['acres_redevelopable'] = 0.0
df_AADevCatwParam['acres_developable'  ] = 0.0

#set acres
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Undevelopable'), 'acres_undevelopable'] = df_AADevCatwParam['Acres']
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Remain'       ), 'acres_remain'       ] = df_AADevCatwParam['Acres']
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Redevelopable'), 'acres_redevelopable'] = df_AADevCatwParam['Acres']
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Developable')  , 'acres_developable'  ] = df_AADevCatwParam['Acres']


#calculate Com and Res SF


#initialize
df_AADevCatwParam['SFcom_undevelopable'   ] = 0.0
df_AADevCatwParam['SFres_undevelopable'   ] = 0.0

df_AADevCatwParam['SFcom_remain'          ] = 0.0
df_AADevCatwParam['SFres_remain'          ] = 0.0

df_AADevCatwParam['SFcom_redeveloped_orig'] = 0.0
df_AADevCatwParam['SFres_redeveloped_orig'] = 0.0

df_AADevCatwParam['SFcom_redeveloped_new' ] = 0.0
df_AADevCatwParam['SFres_redeveloped_new' ] = 0.0

df_AADevCatwParam['SFcom_developed'       ] = 0.0
df_AADevCatwParam['SFres_developed'       ] = 0.0


#set undevelopable sf
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Undevelopable'), 'SFcom_undevelopable'     ] = df_AADevCatwParam['sf_com']
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Undevelopable'), 'SFres_undevelopable'     ] = df_AADevCatwParam['sf_res']

#set remaining sf
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Remain'       ), 'SFcom_remain'            ] = df_AADevCatwParam['sf_com']
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Remain'       ), 'SFres_remain'            ] = df_AADevCatwParam['sf_res']


# SFCom = AcresNoOpenSpace / ((SPLITres/SPLITcom)/(HHperAcre*SFperHH) + (1/(FAR*43560)))
# SFRes = SFCom * SPLITres/SPLITcom
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Redevelopable'), 'SFcom_redeveloped_orig'  ] = df_AADevCatwParam['sf_com']
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Redevelopable'), 'SFres_redeveloped_orig'  ] = df_AADevCatwParam['sf_res']


# temporarily set a very low number for zero to allow calcs to run

df_AADevCatwParam.loc[(df_AADevCatwParam['SFSplitCom'] == 0), 'SFSplitCom'] = 0.00000000001
df_AADevCatwParam.loc[(df_AADevCatwParam['SFSplitRes'] == 0), 'SFSplitRes'] = 0.00000000001

#Split Commercial and Split Residential > 0
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Redevelopable') & (df_AADevCatwParam['SFSplitCom'] >  0) & (df_AADevCatwParam['SFSplitRes'] >  0), 'SFcom_redeveloped_new'   ] = (df_AADevCatwParam['AcresOpenSpaceRemoved'] / ((df_AADevCatwParam['SFSplitRes']/df_AADevCatwParam['SFSplitCom'])/(df_AADevCatwParam['CapacityRes_DUA']*df_AADevCatwParam['SFperHH']) + (1/(df_AADevCatwParam['CapacityCom_FAR']*43560)))).astype('int64')
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Redevelopable') & (df_AADevCatwParam['SFSplitCom'] >  0) & (df_AADevCatwParam['SFSplitRes'] >  0), 'SFres_redeveloped_new'   ] = (df_AADevCatwParam['SFcom_redeveloped_new'] * df_AADevCatwParam['SFSplitRes'] / df_AADevCatwParam['SFSplitCom']).astype('int64')

df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Developable'  ) & (df_AADevCatwParam['SFSplitCom'] >  0) & (df_AADevCatwParam['SFSplitRes'] >  0), 'SFcom_developed'         ] = (df_AADevCatwParam['AcresOpenSpaceRemoved'] / ((df_AADevCatwParam['SFSplitRes']/df_AADevCatwParam['SFSplitCom'])/(df_AADevCatwParam['CapacityRes_DUA']*df_AADevCatwParam['SFperHH']) + (1/(df_AADevCatwParam['CapacityCom_FAR']*43560)))).astype('int64')
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Developable'  ) & (df_AADevCatwParam['SFSplitCom'] >  0) & (df_AADevCatwParam['SFSplitRes'] >  0), 'SFres_developed'         ] = (df_AADevCatwParam['SFcom_developed']       * df_AADevCatwParam['SFSplitRes'] / df_AADevCatwParam['SFSplitCom']).astype('int64')

df_AADevCatwParam.loc[(df_AADevCatwParam['SFSplitCom'] == 0.00000000001), 'SFSplitCom'] = 0
df_AADevCatwParam.loc[(df_AADevCatwParam['SFSplitRes'] == 0.00000000001), 'SFSplitRes'] = 0

#Split Commercial = 0
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Redevelopable') & (df_AADevCatwParam['SFSplitCom'] == 0), 'SFcom_redeveloped_new'   ] = 0
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Redevelopable') & (df_AADevCatwParam['SFSplitCom'] == 0), 'SFres_redeveloped_new'   ] = (df_AADevCatwParam['AcresOpenSpaceRemoved'] * df_AADevCatwParam['SFperHH'] * df_AADevCatwParam['CapacityRes_DUA']).astype('int64')

df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Developable'  ) & (df_AADevCatwParam['SFSplitCom'] == 0), 'SFcom_developed'         ] = 0
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Developable'  ) & (df_AADevCatwParam['SFSplitCom'] == 0), 'SFres_developed'         ] = (df_AADevCatwParam['AcresOpenSpaceRemoved'] * df_AADevCatwParam['SFperHH'] * df_AADevCatwParam['CapacityRes_DUA']).astype('int64')

#Split Residential = 0
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Redevelopable') & (df_AADevCatwParam['SFSplitRes'] == 0), 'SFcom_redeveloped_new'   ] = (df_AADevCatwParam['AcresOpenSpaceRemoved'] * df_AADevCatwParam['CapacityCom_FAR'] * 43560).astype('int64')
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Redevelopable') & (df_AADevCatwParam['SFSplitRes'] == 0), 'SFres_redeveloped_new'   ] = 0

df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Developable'  ) & (df_AADevCatwParam['SFSplitRes'] == 0), 'SFcom_developed'         ] = (df_AADevCatwParam['AcresOpenSpaceRemoved'] * df_AADevCatwParam['CapacityCom_FAR'] * 43560).astype('int64')
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Developable'  ) & (df_AADevCatwParam['SFSplitRes'] == 0), 'SFres_developed'         ] = 0

df_AADevCatwParam

,AreaID,ClassID,county_id,DevCategory,parcel_id,job_spaces,resunits,Acres,sf_res,sf_com,acres_res,acres_com,ClassDescription,SFSplitRes,SFSplitCom,CapacityRes_DUA,CapacityCom_FAR,SFperHH,SFperEmp,PercentOpenSpace,RedevValuePerAcreRes,RedevValuePerAcreCom,RedevAndOr,RedevBldgAgeRes_Low,RedevBldgAgeRes_High,RedevBldgAgeCom_Low,RedevBldgAgeCom_High,RedevProb,ClassOrder,AcresOpenSpaceRemoved,acres_undevelopable,acres_remain,acres_redevelopable,acres_developable,SFcom_undevelopable,SFres_undevelopable,SFcom_remain,SFres_remain,SFcom_redeveloped_orig,SFres_redeveloped_orig,SFcom_redeveloped_new,SFres_redeveloped_new,SFcom_developed,SFres_developed
0,0,a1,0.0,Developable,18.0,0.000000,0.000000,15.567788,0.000000,0.000000,0.000000,0.000000,Single Family A1,1.0,0.0,1.5,0.5,4000,350,0.15,40000,1000000,AND,50,300,20,300,1,1,13.232620,0.000000,0.000000,0.000000,15.567788,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,79395.0
1,0,a1,0.0,Undevelopable,8.0,0.000000,0.000000,14.733847,0.000000,0.000000,0.000000,0.000000,Single Family A1,1.0,0.0,1.5,0.5,4000,350,0.15,40000,1000000,AND,50,300,20,300,1,1,12.523770,14.733847,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
2,0,a1,1.0,Undevelopable,1.0,0.000000,0.000000,0.108287,0.000000,0.000000,0.000000,0.000000,Single Family A1,1.0,0.0,1.5,0.5,4000,350,0.15,40000,1000000,AND,50,300,20,300,1,1,0.092044,0.108287,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
3,0,a1,2.0,Developable,102.0,0.000000,0.000000,234.281314,0.000000,0.000000,0.000000,0.000000,Single Family A1,1.0,0.0,1.5,0.5,4000,350,0.15,40000,1000000,AND,50,300,20,300,1,1,199.139117,0.000000,0.000000,0.000000,234.281314,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,1194834.0
4,0,a1,2.0,Redevelopable,9.0,1.470310,9.117647,84.854662,13154.026381,1433.552632,83.921666,0.932996,Single Family A1,1.0,0.0,1.5,0.5,4000,350,0.15,40000,1000000,AND,50,300,20,300,1,1,72.126463,0.000000,0.000000,84.854662,0.000000,0.000000,0.0,0.000000,0.000000,1433.552632,13154.026381,0.0,432758.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2357,77,f2,1.0,Undevelopable,550.0,13.989540,0.000000,15.171574,0.000000,5232.241785,0.000000,0.000000,Multifamily Suburban,0.7,0.3,30.0,1.0,1500,300,0.10,1500000,1200000,AND,60,120,20,120,1,27,13.654417,15.171574,0.000000,0.000000,0.000000,5232.241785,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
2358,77,g2,1.0,Developable,39.0,0.000000,0.000000,94.609768,0.000000,0.000000,0.000000,0.000000,Mixed-Use Suburban,0.7,0.3,30.0,1.0,1500,300,0.10,1500000,1200000,AND,60,120,20,120,1,31,85.148791,0.000000,0.000000,0.000000,94.609768,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,1138220.0,2655846.0
2359,77,g2,1.0,Redevelopable,2.0,0.000109,0.999174,0.320516,1187.019228,0.040774,0.320457,0.000059,Mixed-Use Suburban,0.7,0.3,30.0,1.0,1500,300,0.10,1500000,1200000,AND,60,120,20,120,1,31,0.288464,0.000000,0.000000,0.320516,0.000000,0.000000,0.0,0.000000,0.000000,0.040774,1187.019228,3856.0,8997.0,0.0,0.0
2360,77,g2,1.0,Remain,14.0,0.002325,6.064735,5.687062,15878.834261,1.471514,5.682825,0.004237,Mixed-Use Suburban,0.7,0.3,30.0,1.0,1500,300,0.10,1500000,1200000,AND,60,120,20,120,1,31,5.118356,0.000000,5.687062,0.000000,0.000000,0.000000,0.0,1.471514,15878.834261,0.000000,0.000000,0.0,0.0,0.0,0.0


In [20]:
df1 = df_AADevCatwParam[df_AADevCatwParam.isna().any(axis=1)]
display(df1)

df2 = df_AADevCatwParam[(df_AADevCatwParam == np.inf).any(axis=1)]
display (df2)

,AreaID,ClassID,county_id,DevCategory,parcel_id,job_spaces,resunits,Acres,sf_res,sf_com,acres_res,acres_com,ClassDescription,SFSplitRes,SFSplitCom,CapacityRes_DUA,CapacityCom_FAR,SFperHH,SFperEmp,PercentOpenSpace,RedevValuePerAcreRes,RedevValuePerAcreCom,RedevAndOr,RedevBldgAgeRes_Low,RedevBldgAgeRes_High,RedevBldgAgeCom_Low,RedevBldgAgeCom_High,RedevProb,ClassOrder,AcresOpenSpaceRemoved,acres_undevelopable,acres_remain,acres_redevelopable,acres_developable,SFcom_undevelopable,SFres_undevelopable,SFcom_remain,SFres_remain,SFcom_redeveloped_orig,SFres_redeveloped_orig,SFcom_redeveloped_new,SFres_redeveloped_new,SFcom_developed,SFres_developed


,AreaID,ClassID,county_id,DevCategory,parcel_id,job_spaces,resunits,Acres,sf_res,sf_com,acres_res,acres_com,ClassDescription,SFSplitRes,SFSplitCom,CapacityRes_DUA,CapacityCom_FAR,SFperHH,SFperEmp,PercentOpenSpace,RedevValuePerAcreRes,RedevValuePerAcreCom,RedevAndOr,RedevBldgAgeRes_Low,RedevBldgAgeRes_High,RedevBldgAgeCom_Low,RedevBldgAgeCom_High,RedevProb,ClassOrder,AcresOpenSpaceRemoved,acres_undevelopable,acres_remain,acres_redevelopable,acres_developable,SFcom_undevelopable,SFres_undevelopable,SFcom_remain,SFres_remain,SFcom_redeveloped_orig,SFres_redeveloped_orig,SFcom_redeveloped_new,SFres_redeveloped_new,SFcom_developed,SFres_developed


In [21]:
#calculate jobs and HH

#initialize
df_AADevCatwParam['job_spaces_undevelopable'   ] = 0.0
df_AADevCatwParam['job_spaces_remain'          ] = 0.0
df_AADevCatwParam['job_spaces_redeveloped_orig'] = 0.0
df_AADevCatwParam['job_spaces_redeveloped_new' ] = 0.0
df_AADevCatwParam['job_spaces_developed'       ] = 0.0

df_AADevCatwParam['resunits_undevelopable'     ] = 0.0
df_AADevCatwParam['resunits_remain'            ] = 0.0
df_AADevCatwParam['resunits_redeveloped_orig'  ] = 0.0
df_AADevCatwParam['resunits_redeveloped_new'   ] = 0.0
df_AADevCatwParam['resunits_developed'         ] = 0.0

#set undevelopable sf
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Undevelopable'), 'job_spaces_undevelopable'   ] =  df_AADevCatwParam['job_spaces'           ]
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Remain'       ), 'job_spaces_remain'          ] =  df_AADevCatwParam['job_spaces'           ]
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Redevelopable'), 'job_spaces_redeveloped_orig'] =  df_AADevCatwParam['job_spaces'           ]
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Redevelopable'), 'job_spaces_redeveloped_new' ] = (df_AADevCatwParam['SFcom_redeveloped_new'] / df_AADevCatwParam['SFperEmp']).astype('int64')
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Developable'  ), 'job_spaces_developed'       ] = (df_AADevCatwParam['SFcom_developed'      ] / df_AADevCatwParam['SFperEmp']).astype('int64')

df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Undevelopable'), 'resunits_undevelopable'     ] =  df_AADevCatwParam['resunits'             ]
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Remain'       ), 'resunits_remain'            ] =  df_AADevCatwParam['resunits'             ]
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Redevelopable'), 'resunits_redeveloped_orig'  ] =  df_AADevCatwParam['resunits'             ]
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Redevelopable'), 'resunits_redeveloped_new'   ] = (df_AADevCatwParam['SFcom_redeveloped_new'] / df_AADevCatwParam['SFperHH' ]).astype('int64')
df_AADevCatwParam.loc[(df_AADevCatwParam['DevCategory'] == 'Developable'  ), 'resunits_developed'         ] = (df_AADevCatwParam['SFcom_developed'      ] / df_AADevCatwParam['SFperHH' ]).astype('int64')


df_AADevCatwParam

,AreaID,ClassID,county_id,DevCategory,parcel_id,job_spaces,resunits,Acres,sf_res,sf_com,acres_res,acres_com,ClassDescription,SFSplitRes,SFSplitCom,CapacityRes_DUA,CapacityCom_FAR,SFperHH,SFperEmp,PercentOpenSpace,RedevValuePerAcreRes,RedevValuePerAcreCom,RedevAndOr,RedevBldgAgeRes_Low,RedevBldgAgeRes_High,RedevBldgAgeCom_Low,RedevBldgAgeCom_High,RedevProb,ClassOrder,AcresOpenSpaceRemoved,acres_undevelopable,acres_remain,acres_redevelopable,acres_developable,SFcom_undevelopable,SFres_undevelopable,SFcom_remain,SFres_remain,SFcom_redeveloped_orig,SFres_redeveloped_orig,SFcom_redeveloped_new,SFres_redeveloped_new,SFcom_developed,SFres_developed,job_spaces_undevelopable,job_spaces_remain,job_spaces_redeveloped_orig,job_spaces_redeveloped_new,job_spaces_developed,resunits_undevelopable,resunits_remain,resunits_redeveloped_orig,resunits_redeveloped_new,resunits_developed
0,0,a1,0.0,Developable,18.0,0.000000,0.000000,15.567788,0.000000,0.000000,0.000000,0.000000,Single Family A1,1.0,0.0,1.5,0.5,4000,350,0.15,40000,1000000,AND,50,300,20,300,1,1,13.232620,0.000000,0.000000,0.000000,15.567788,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,79395.0,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
1,0,a1,0.0,Undevelopable,8.0,0.000000,0.000000,14.733847,0.000000,0.000000,0.000000,0.000000,Single Family A1,1.0,0.0,1.5,0.5,4000,350,0.15,40000,1000000,AND,50,300,20,300,1,1,12.523770,14.733847,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
2,0,a1,1.0,Undevelopable,1.0,0.000000,0.000000,0.108287,0.000000,0.000000,0.000000,0.000000,Single Family A1,1.0,0.0,1.5,0.5,4000,350,0.15,40000,1000000,AND,50,300,20,300,1,1,0.092044,0.108287,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
3,0,a1,2.0,Developable,102.0,0.000000,0.000000,234.281314,0.000000,0.000000,0.000000,0.000000,Single Family A1,1.0,0.0,1.5,0.5,4000,350,0.15,40000,1000000,AND,50,300,20,300,1,1,199.139117,0.000000,0.000000,0.000000,234.281314,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,1194834.0,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
4,0,a1,2.0,Redevelopable,9.0,1.470310,9.117647,84.854662,13154.026381,1433.552632,83.921666,0.932996,Single Family A1,1.0,0.0,1.5,0.5,4000,350,0.15,40000,1000000,AND,50,300,20,300,1,1,72.126463,0.000000,0.000000,84.854662,0.000000,0.000000,0.0,0.000000,0.000000,1433.552632,13154.026381,0.0,432758.0,0.0,0.0,0.00000,0.000000,1.470310,0.0,0.0,0.0,0.000000,9.117647,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2357,77,f2,1.0,Undevelopable,550.0,13.989540,0.000000,15.171574,0.000000,5232.241785,0.000000,0.000000,Multifamily Suburban,0.7,0.3,30.0,1.0,1500,300,0.10,1500000,1200000,AND,60,120,20,120,1,27,13.654417,15.171574,0.000000,0.000000,0.000000,5232.241785,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,13.98954,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
2358,77,g2,1.0,Developable,39.0,0.000000,0.000000,94.609768,0.000000,0.000000,0.000000,0.000000,Mixed-Use Suburban,0.7,0.3,30.0,1.0,1500,300,0.10,1500000,1200000,AND,60,120,20,120,1,31,85.148791,0.000000,0.000000,0.000000,94.609768,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,1138220.0,2655846.0,0.00000,0.000000,0.000000,0.0,3794.0,0.0,0.000000,0.000000,0.0,758.0
2359,77,g2,1.0,Redevelopable,2.0,0.000109,0.999174,0.320516,1187.019228,0.040774,0.320457,0.000059,Mixed-Use Suburban,0.7,0.3,30.0,1.0,1500,300,0.10,1500000,1200000,AND,60,120,20,120,1,31,0.288464,0.000000,0.000000,0.320516,0.000000,0.000000,0.0,0.000000,0.000000,0.040774,1187.019228,3856.0,8997.0,0.0,0.0,0.00000,0.000000,0.000109,12.0,0.0,0.0,0.000000,0.9991

In [22]:
#calculate totals

df_AASummary = df_AADevCatwParam.groupby(['county_id','AreaID','ClassID','ClassDescription','ClassOrder'], as_index=False).agg({"DevCategory":[np.size], "parcel_id":[np.sum], "Acres":[np.sum], "acres_undevelopable":[np.sum], "acres_remain":[np.sum], "acres_redevelopable":[np.sum], "acres_developable":[np.sum], "SFcom_undevelopable":[np.sum], "SFres_undevelopable":[np.sum], "SFcom_remain":[np.sum], "SFres_remain":[np.sum], "SFcom_redeveloped_orig":[np.sum], "SFres_redeveloped_orig":[np.sum], "SFcom_redeveloped_new":[np.sum], "SFres_redeveloped_new":[np.sum], "SFcom_developed":[np.sum], "SFres_developed":[np.sum], "job_spaces_undevelopable":[np.sum], "job_spaces_remain":[np.sum], "job_spaces_redeveloped_orig":[np.sum], "job_spaces_redeveloped_new":[np.sum], "job_spaces_developed":[np.sum], "resunits_undevelopable":[np.sum], "resunits_remain":[np.sum], "resunits_redeveloped_orig":[np.sum], "resunits_redeveloped_new":[np.sum], "resunits_developed":[np.sum]})
df_AASummary.columns = df_AASummary.columns.droplevel(1)

df_AASummaryTotals = df_AASummary.copy()

df_AASummaryTotals['job_spaces_orig'  ] = (df_AASummaryTotals['job_spaces_undevelopable'] + df_AASummaryTotals['job_spaces_remain'] + df_AASummaryTotals['job_spaces_redeveloped_orig']).astype('int64')
df_AASummaryTotals['job_spaces_new'   ] = (df_AASummaryTotals['job_spaces_undevelopable'] + df_AASummaryTotals['job_spaces_remain'] + df_AASummaryTotals['job_spaces_redeveloped_new' ] + df_AASummaryTotals['job_spaces_developed']).astype('int64')
df_AASummaryTotals['job_spaces_change'] = (df_AASummaryTotals['job_spaces_new'          ] - df_AASummaryTotals['job_spaces_orig'  ]).round(0)

df_AASummaryTotals['resunits_orig'    ] = (df_AASummaryTotals['resunits_undevelopable'  ] + df_AASummaryTotals['resunits_remain'  ] + df_AASummaryTotals['resunits_redeveloped_orig'  ]).astype('int64')
df_AASummaryTotals['resunits_new'     ] = (df_AASummaryTotals['resunits_undevelopable'  ] + df_AASummaryTotals['resunits_remain'  ] + df_AASummaryTotals['resunits_redeveloped_new'   ] + df_AASummaryTotals['resunits_developed'  ]).astype('int64')
df_AASummaryTotals['resunits_change'  ] = (df_AASummaryTotals['resunits_new'            ] - df_AASummaryTotals['resunits_orig'    ]).round(0)

df_AASummaryTotals = df_AASummaryTotals[['county_id','AreaID','ClassID','ClassDescription','ClassOrder','parcel_id','Acres','acres_redevelopable','acres_developable','job_spaces_orig','job_spaces_new','job_spaces_change','resunits_orig','resunits_new','resunits_change','job_spaces_undevelopable','job_spaces_remain','job_spaces_redeveloped_orig','job_spaces_redeveloped_new','job_spaces_developed','resunits_undevelopable','resunits_remain','resunits_redeveloped_orig','resunits_redeveloped_new','resunits_developed']]

display(df_AASummaryTotals.sum())

df_AASummaryTotals['density_hhemp_orig'  ] = ((df_AASummaryTotals['job_spaces_orig'  ] + df_AASummaryTotals['resunits_orig'     ]) / df_AASummaryTotals['Acres']).round(1)
df_AASummaryTotals['density_hhemp_new'   ] = ((df_AASummaryTotals['job_spaces_new'   ] + df_AASummaryTotals['resunits_new'      ]) / df_AASummaryTotals['Acres']).round(1)
df_AASummaryTotals['density_hhemp_change'] =  (df_AASummaryTotals['density_hhemp_new'] - df_AASummaryTotals['density_hhemp_orig']).round(1)


county_id                                                                   1746
AreaID                                                                     27238
ClassID                        a1a3a4a5c1a2a4b4b4a1a4d3a1a5b4b5b1c1d1b3a1c3a2...
ClassDescription               Single Family A1Single Family A3Single Family ...
ClassOrder                                                                 10988
parcel_id                                                                 896222
Acres                                                                     306172
acres_redevelopable                                                      24547.6
acres_developable                                                        89707.7
job_spaces_orig                                                           335629
job_spaces_new                                                           1905788
job_spaces_change                                                        1570159
resunits_orig               

In [23]:
#create map centered on Salt Lake
map_parcels_areas_den = gis.map('Salt Lake')

sdf_AADevCatwParam = pd.DataFrame.merge(sdf_AAid, df_AASummaryTotals, on=['AreaID','ClassID'], how='left')

#expression to classify employment growth
arcade_expression_denchange = ("var v = $feature.density_hhemp_change;"
                                     "if      (v<0    ) { return 'class0'; }"
                                     "if      (v<20   ) { return 'class1'; }"
                                     "else if (v<50   ) { return 'class2'; }"
                                     "else if (v<100  ) { return 'class3'; }"
                                     "else              { return 'class4'; }")

#symbology for enrollment classes
uv_denchange = [
                #{"value":"class0", "label":"Density Decline", "symbol":{"type":"esriSFS","color":[  0,  0,  0,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
                {"value":"class1", "label":"0 to 20"      , "symbol":{"type":"esriSFS","color":[130,165,217,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
                {"value":"class2", "label":"20 to 50"     , "symbol":{"type":"esriSFS","color":[140,125,164,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
                {"value":"class3", "label":"50 to 100"    , "symbol":{"type":"esriSFS","color":[149, 85,111,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
                {"value":"class4", "label":"More than 100", "symbol":{"type":"esriSFS","color":[168,  4,  4,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}}
               ]


#define sdf layer
sdf_AADevCatwParam.spatial.plot(map_widget = map_parcels_areas_den,
                                renderer_type='u-a', #'u-a' stands for uniqe value with arcade expression
                                unique_values=uv_denchange,
                                arcade_expression=arcade_expression_denchange,
                                default_symbol="" #don't include an 'other' category
                                )

#define map characteristics
map_parcels_areas_den.layout.height='500px'
map_parcels_areas_den.legend=True

#map title
display(Markdown('<h2><center>Density Change by Analysis Area</center></h2>'))

#display map
map_parcels_areas_den

<h2><center>Density Change by Analysis Area</center></h2>

MapView(layout=Layout(height='500px', width='100%'), legend=True)

# Area Summary

In [24]:
df_AreaSummary = df_AASummaryTotals.groupby(['AreaID'], as_index=False).agg({"Acres":[np.sum],"job_spaces_orig":[np.sum],"job_spaces_new":[np.sum],"job_spaces_change":[np.sum],"resunits_orig":[np.sum],"resunits_new":[np.sum],"resunits_change":[np.sum]})
df_AreaSummary.columns = df_AreaSummary.columns.droplevel(1)

#read in csv with area names
df_AreaNames = pd.read_csv(AnalysisAreasTable)

#merge area names with area summary
df_AreaSummary_wNames = pd.DataFrame.merge(df_AreaNames, df_AreaSummary, on="AreaID")

#add total line to area summary with names
df_AreaSummary_wNames_wTotals = df_AreaSummary_wNames.append(df_AreaSummary_wNames.sum(numeric_only=True).rename('Total'))

#recalculate densities, round to nearest whole number (calculate before rounding of jobs/units)
df_AreaSummary_wNames_wTotals['density_hhemp_orig'  ] = ((df_AreaSummary_wNames_wTotals['job_spaces_orig'  ] + df_AreaSummary_wNames_wTotals['resunits_orig'     ]) / df_AreaSummary_wNames_wTotals['Acres']).round(0)
df_AreaSummary_wNames_wTotals['density_hhemp_new'   ] = ((df_AreaSummary_wNames_wTotals['job_spaces_new'   ] + df_AreaSummary_wNames_wTotals['resunits_new'      ]) / df_AreaSummary_wNames_wTotals['Acres']).round(0)
df_AreaSummary_wNames_wTotals['density_hhemp_change'] =  (df_AreaSummary_wNames_wTotals['density_hhemp_new'] - df_AreaSummary_wNames_wTotals['density_hhemp_orig']).round(0)

#recalculate based on rounded values, rounded to nearest hundred
df_AreaSummary_wNames_wTotals['job_spaces_orig'     ] = (df_AreaSummary_wNames_wTotals['job_spaces_orig'   ]).round(-2)
df_AreaSummary_wNames_wTotals['job_spaces_new'      ] = (df_AreaSummary_wNames_wTotals['job_spaces_new'    ]).round(-2)
df_AreaSummary_wNames_wTotals['job_spaces_change'   ] =  df_AreaSummary_wNames_wTotals['job_spaces_new'    ] - df_AreaSummary_wNames_wTotals['job_spaces_orig'] 

df_AreaSummary_wNames_wTotals['resunits_orig'       ] = (df_AreaSummary_wNames_wTotals['resunits_orig'     ]).round(-2)
df_AreaSummary_wNames_wTotals['resunits_new'        ] = (df_AreaSummary_wNames_wTotals['resunits_new'      ]).round(-2)
df_AreaSummary_wNames_wTotals['resunits_change'     ] =  df_AreaSummary_wNames_wTotals['resunits_new'      ] - df_AreaSummary_wNames_wTotals['resunits_orig'] 

#change index to Area ID
df_AreaSummary_wNames_wTotals = df_AreaSummary_wNames_wTotals.set_index(['AreaID'])

#display table with numeric styling
display(df_AreaSummary_wNames_wTotals.style.format({"Acres":"{:,.1f}","job_spaces_orig":"{:,.0f}","job_spaces_new":"{:,.0f}","job_spaces_change":"{:,.0f}","resunits_orig":"{:,.0f}","resunits_new":"{:,.0f}","resunits_change":"{:,.0f}","density_hhemp_orig":"{:,.0f}","density_hhemp_new":"{:,.0f}","density_hhemp_change":"{:,.0f}"}))

#export to csv
df_AreaSummary_wNames_wTotals.to_csv(os.path.join(dir_results, r'SummaryTable_Area_wTotals.csv'))

,AnalysisAreaName,Acres,job_spaces_orig,job_spaces_new,job_spaces_change,resunits_orig,resunits_new,resunits_change,density_hhemp_orig,density_hhemp_new,density_hhemp_change
AreaID,,,,,,,,,,,
0.0,nan,"30,383.7","2,500","4,800","2,300","6,300","5,900",-400,0,0,0
1.0,Alpine,"3,803.8",600,600,0,"2,700","2,600",-100,1,1,0
3.0,American Fork,"3,993.6","1,100","1,700",600,"8,200","8,000",-200,2,2,0
4.0,Bluffdale,"3,522.4",100,600,500,"2,300","2,300",0,1,1,0
5.0,Bountiful,"6,180.7","7,400","9,200","1,800","15,700","13,100","-2,600",4,4,0
7.0,Cedar Fort,453.1,0,0,0,100,100,0,0,0,0
8.0,Cedar Hills,"1,023.2",100,100,0,"2,600","2,500",-100,3,3,0
9.0,Centerville,"1,327.6",500,"5,300","4,800","5,000","5,700",700,4,8,4
10.0,Clearfield,"2,128.9","4,400","52,600","48,200","9,000","17,000","8,000",6,33,27


# Class Summary

In [26]:
#Class table is aggregate of df_AASummaryTotals
df_ClassSummaryTable = df_AASummaryTotals.groupby(['ClassOrder','ClassDescription'], as_index=False).agg({"Acres":[np.sum],"acres_redevelopable":[np.sum],"acres_developable":[np.sum],"job_spaces_orig":[np.sum],"job_spaces_new":[np.sum],"job_spaces_change":[np.sum],"resunits_orig":[np.sum],"resunits_new":[np.sum],"resunits_change":[np.sum], "job_spaces_undevelopable":[np.sum], "job_spaces_remain":[np.sum], "job_spaces_redeveloped_orig":[np.sum], "job_spaces_redeveloped_new":[np.sum], "job_spaces_developed":[np.sum], "resunits_undevelopable":[np.sum], "resunits_remain":[np.sum], "resunits_redeveloped_orig":[np.sum], "resunits_redeveloped_new":[np.sum], "resunits_developed":[np.sum]})

#drop aggregate description from column name (sum, etc)
df_ClassSummaryTable.columns = df_ClassSummaryTable.columns.droplevel(1)

#add totals row
df_ClassSummaryTable_wTotals = df_ClassSummaryTable.append(df_ClassSummaryTable.sum(numeric_only=True).rename('Total'))

#recalculate densities - calculate densities before rounding
df_ClassSummaryTable_wTotals['density_hhemp_orig'  ] = ((df_ClassSummaryTable_wTotals['job_spaces_orig'  ] + df_ClassSummaryTable_wTotals['resunits_orig'     ]) / df_ClassSummaryTable_wTotals['Acres']).round(0)
df_ClassSummaryTable_wTotals['density_hhemp_new'   ] = ((df_ClassSummaryTable_wTotals['job_spaces_new'   ] + df_ClassSummaryTable_wTotals['resunits_new'      ]) / df_ClassSummaryTable_wTotals['Acres']).round(0)
df_ClassSummaryTable_wTotals['density_hhemp_change'] =  (df_ClassSummaryTable_wTotals['density_hhemp_new'] - df_ClassSummaryTable_wTotals['density_hhemp_orig']).round(0)

#calculate rounded values
df_ClassSummaryTable_wTotals['job_spaces_orig'     ] =  (df_ClassSummaryTable_wTotals['job_spaces_orig'  ]).round(-2)
df_ClassSummaryTable_wTotals['job_spaces_new'      ] =  (df_ClassSummaryTable_wTotals['job_spaces_new'   ]).round(-2)
df_ClassSummaryTable_wTotals['job_spaces_change'   ] =   df_ClassSummaryTable_wTotals['job_spaces_new'   ] - df_ClassSummaryTable_wTotals['job_spaces_orig'] 

df_ClassSummaryTable_wTotals['resunits_orig'       ] =  (df_ClassSummaryTable_wTotals['resunits_orig'    ]).round(-2)
df_ClassSummaryTable_wTotals['resunits_new'        ] =  (df_ClassSummaryTable_wTotals['resunits_new'     ]).round(-2)
df_ClassSummaryTable_wTotals['resunits_change'     ] =   df_ClassSummaryTable_wTotals['resunits_new'     ] - df_ClassSummaryTable_wTotals['resunits_orig'] 

#fix last row name to be (HARD CODED CLASS ORDER-FIX!!)
df_ClassSummaryTable_wTotals.loc[(df_ClassSummaryTable_wTotals['ClassOrder'] == 28), 'ClassDescription'] = 'Total'
df_ClassSummaryTable_wTotals.loc[(df_ClassSummaryTable_wTotals['ClassOrder'] == 28), 'ClassOrder'] = 8

#set index to class order
df_ClassSummaryTable_wTotals = df_ClassSummaryTable_wTotals.set_index(['ClassOrder'])

df_ClassSummaryTable_wTotals['job_spaces_redev_add'       ] = df_ClassSummaryTable_wTotals['job_spaces_redeveloped_new'] -  df_ClassSummaryTable_wTotals['job_spaces_redeveloped_orig']
df_ClassSummaryTable_wTotals['resunits_redev_add'         ] = df_ClassSummaryTable_wTotals['resunits_redeveloped_new'  ] -  df_ClassSummaryTable_wTotals['resunits_redeveloped_orig'  ]

df_ClassSummaryTable_wTotals['job_spaces_redev_percentadd'] = df_ClassSummaryTable_wTotals['job_spaces_redev_add'      ] / (df_ClassSummaryTable_wTotals['job_spaces_redev_add'       ] + df_ClassSummaryTable_wTotals['job_spaces_developed'])
df_ClassSummaryTable_wTotals['job_spaces_dev_percentadd'  ] = df_ClassSummaryTable_wTotals['job_spaces_developed'      ] / (df_ClassSummaryTable_wTotals['job_spaces_redev_add'       ] + df_ClassSummaryTable_wTotals['job_spaces_developed'])

df_ClassSummaryTable_wTotals['resunits_redev_percentadd'  ] = df_ClassSummaryTable_wTotals['resunits_redev_add'        ] / (df_ClassSummaryTable_wTotals['resunits_redev_add'         ] + df_ClassSummaryTable_wTotals['resunits_developed'  ])
df_ClassSummaryTable_wTotals['resunits_dev_percentadd'    ] = df_ClassSummaryTable_wTotals['resunits_developed'        ] / (df_ClassSummaryTable_wTotals['resunits_redev_add'         ] + df_ClassSummaryTable_wTotals['resunits_developed'  ])

df_ClassSummaryTable_wTotals['Acres_Redev_Percent'        ] =  df_ClassSummaryTable_wTotals['acres_redevelopable'      ]                                                                / df_ClassSummaryTable_wTotals['Acres']
df_ClassSummaryTable_wTotals['Acres_Dev_Percent'          ] =  df_ClassSummaryTable_wTotals['acres_developable'        ]                                                                / df_ClassSummaryTable_wTotals['Acres']

df_ClassSummaryTable_wTotals['Acres_Unchanged_Percent'    ] = 1 - df_ClassSummaryTable_wTotals['Acres_Dev_Percent'] - df_ClassSummaryTable_wTotals['Acres_Redev_Percent']

df_ClassSummaryTable_wTotals = df_ClassSummaryTable_wTotals[['ClassDescription','Acres','job_spaces_orig','job_spaces_new','job_spaces_change','resunits_orig','resunits_new','resunits_change','density_hhemp_orig','density_hhemp_new','density_hhemp_change','Acres_Unchanged_Percent','Acres_Redev_Percent','Acres_Dev_Percent','job_spaces_redev_percentadd','job_spaces_dev_percentadd','resunits_redev_percentadd','resunits_dev_percentadd']]

#display with formatted numbers
#display(df_ClassSummaryTable_wTotals.style.format({"Acres":"{:,.0f}","job_spaces_orig":"{:,.0f}","job_spaces_new":"{:,.0f}","job_spaces_change":"{:,.0f}","resunits_orig":"{:,.0f}","resunits_new":"{:,.0f}","resunits_change":"{:,.0f}","density_hhemp_orig":"{:,.0f}","density_hhemp_new":"{:,.0f}","density_hhemp_change":"{:,.0f}","Acres_Unchanged_Percent":"{:,.0%}","Acres_Redev_Percent":"{:,.0%}","Acres_Dev_Percent":"{:,.0%}","job_spaces_redev_percentadd":"{:,.0%}","job_spaces_dev_percentadd":"{:,.0%}","resunits_redev_percentadd":"{:,.0%}","resunits_dev_percentadd":"{:,.0%}"}))

#export to csv
df_ClassSummaryTable_wTotals.to_csv(os.path.join(dir_results, r'SummaryTable_Class_wTotals.csv'))

In [29]:
df_AASummaryTotals
#df_ClassSummaryTable_wTotals

,county_id,AreaID,ClassID,ClassDescription,ClassOrder,parcel_id,Acres,acres_redevelopable,acres_developable,job_spaces_orig,job_spaces_new,job_spaces_change,resunits_orig,resunits_new,resunits_change,job_spaces_undevelopable,job_spaces_remain,job_spaces_redeveloped_orig,job_spaces_redeveloped_new,job_spaces_developed,resunits_undevelopable,resunits_remain,resunits_redeveloped_orig,resunits_redeveloped_new,resunits_developed,density_hhemp_orig,density_hhemp_new,density_hhemp_change
0,0.0,0,a1,Single Family A1,1,26.0,30.301635,0.000000,15.567788,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
1,0.0,0,a3,Single Family A3,3,1.0,1.997394,0.000000,0.000000,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
2,0.0,0,a4,Single Family A4,4,3.0,7.459026,0.000000,0.000000,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
3,0.0,0,a5,Single Family A5,5,3.0,2.734494,0.000000,0.000000,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
4,0.0,0,c1,Single Family C1,11,4.0,0.684886,0.000000,0.000000,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
713,4.0,68,f2,Multifamily Suburban,27,90.0,128.241932,0.000000,119.331322,0,4785,4785,0,957,957,0.0,0.0,0.0,0.0,4785.0,0.0,0.000000,0.000000,0.0,957.0,0.0,44.8,44.8
714,4.0,68,g3,Mixed-Use City/TOD,32,221.0,301.336862,0.000000,283.142131,0,60492,60492,0,10082,10082,0.0,0.0,0.0,0.0,60492.0,0.0,0.000000,0.000000,0.0,10082.0,0.0,234.2,234.2
715,4.0,76,a1,Single Family A1,1,421.0,499.477617,24.631539,397.194496,0,0,0,19,18,-1,0.0,0.0,0.0,0.0,0.0,0.0,18.092913,0.999470,0.0,0.0,0.0,0.0,0.0
716,4.0,76,a2,Single Family A2,2,1695.0,863.727177,2.496990,369.171040,0,0,0,367,366,-1,0.0,0.0,0.0,0.0,0.0,0.0,366.660632,0.999469,0.0,0.0,0.4,0.4,0.0


# Summary by County

In [31]:
df_CountySummary = df_AASummaryTotals.groupby(['county_id'], as_index=False).agg({"Acres":[np.sum],"job_spaces_orig":[np.sum],"job_spaces_new":[np.sum],"job_spaces_change":[np.sum],"resunits_orig":[np.sum],"resunits_new":[np.sum],"resunits_change":[np.sum]})
df_CountySummary.columns = df_CountySummary.columns.droplevel(1)

#read in csv with area names
df_CountyNames = pd.read_csv(CountiesTable)

#merge area names with area summary
df_CountySummary_wNames = pd.DataFrame.merge(df_CountyNames, df_CountySummary, on="county_id")

#add total line to area summary with names
df_CountySummary_wNames_wTotals = df_CountySummary_wNames.append(df_CountySummary_wNames.sum(numeric_only=True).rename('Total'))

#recalculate densities, round to nearest whole number (calculate before rounding of jobs/units)
df_CountySummary_wNames_wTotals['density_hhemp_orig'  ] = ((df_CountySummary_wNames_wTotals['job_spaces_orig'  ] + df_CountySummary_wNames_wTotals['resunits_orig'     ]) / df_CountySummary_wNames_wTotals['Acres']).round(0)
df_CountySummary_wNames_wTotals['density_hhemp_new'   ] = ((df_CountySummary_wNames_wTotals['job_spaces_new'   ] + df_CountySummary_wNames_wTotals['resunits_new'      ]) / df_CountySummary_wNames_wTotals['Acres']).round(0)
df_CountySummary_wNames_wTotals['density_hhemp_change'] =  (df_CountySummary_wNames_wTotals['density_hhemp_new'] - df_CountySummary_wNames_wTotals['density_hhemp_orig']).round(0)

#recalculate based on rounded values, rounded to nearest hundred
df_CountySummary_wNames_wTotals['job_spaces_orig'     ] = (df_CountySummary_wNames_wTotals['job_spaces_orig'   ]).round(-2)
df_CountySummary_wNames_wTotals['job_spaces_new'      ] = (df_CountySummary_wNames_wTotals['job_spaces_new'    ]).round(-2)
df_CountySummary_wNames_wTotals['job_spaces_change'   ] =  df_CountySummary_wNames_wTotals['job_spaces_new'    ] - df_CountySummary_wNames_wTotals['job_spaces_orig'] 

df_CountySummary_wNames_wTotals['resunits_orig'       ] = (df_CountySummary_wNames_wTotals['resunits_orig'     ]).round(-2)
df_CountySummary_wNames_wTotals['resunits_new'        ] = (df_CountySummary_wNames_wTotals['resunits_new'      ]).round(-2)
df_CountySummary_wNames_wTotals['resunits_change'     ] =  df_CountySummary_wNames_wTotals['resunits_new'      ] - df_CountySummary_wNames_wTotals['resunits_orig'] 

#change index to Area ID
df_CountySummary_wNames_wTotals = df_CountySummary_wNames_wTotals.set_index(['CO_ORDER'])

#display table with numeric styling
#display(df_CountySummary_wNames_wTotals.style.format({"Acres":"{:,.1f}","job_spaces_orig":"{:,.0f}","job_spaces_new":"{:,.0f}","job_spaces_change":"{:,.0f}","resunits_orig":"{:,.0f}","resunits_new":"{:,.0f}","resunits_change":"{:,.0f}","density_hhemp_orig":"{:,.0f}","density_hhemp_new":"{:,.0f}","density_hhemp_change":"{:,.0f}"}))

#export to csv
df_CountySummary_wNames_wTotals.to_csv(os.path.join(dir_results, r'SummaryTable_County_wTotals.csv'))

In [32]:
df_CountySummary_wNames_wTotals

,county_id,CO_NAME,Acres,job_spaces_orig,job_spaces_new,job_spaces_change,resunits_orig,resunits_new,resunits_change,density_hhemp_orig,density_hhemp_new,density_hhemp_change
CO_ORDER,,,,,,,,,,,,
1.0,3.0,Weber,60710.576506,40800.0,173800.0,133000.0,87600.0,104300.0,16700.0,2.0,5.0,3.0
2.0,1.0,Davis,41697.325956,40800.0,238400.0,197600.0,83800.0,113400.0,29600.0,3.0,8.0,5.0
3.0,2.0,Salt Lake,93875.517583,195600.0,1145500.0,949900.0,360800.0,475500.0,114700.0,6.0,17.0,11.0
4.0,4.0,Utah,108347.632644,58400.0,348000.0,289600.0,137600.0,181600.0,44000.0,2.0,5.0,3.0
10.0,10.0,NaN,304631.052690,335600.0,1905800.0,1570200.0,669800.0,874900.0,205100.0,3.0,9.0,6.0


# Display HH/Emp Growth by Analysis Areas

In [33]:
df_AnalysisAreas = pd.read_csv(AnalysisAreasTable)

sdf_AnalysisAreas = pd.DataFrame.merge(sdf_AAid, df_AnalysisAreas, on='AreaID')

sdf_AANewHHEmp = pd.DataFrame.merge(sdf_AnalysisAreas, df_AASummaryTotals, on=['AreaID','ClassID'])
sdf_AANewHHEmp

,FID,Shape_Leng,Shape_Area,AreaID,ClassID,SHAPE,AnalysisAreaName,county_id,ClassDescription,ClassOrder,parcel_id,Acres,acres_redevelopable,acres_developable,job_spaces_orig,job_spaces_new,job_spaces_change,resunits_orig,resunits_new,resunits_change,job_spaces_undevelopable,job_spaces_remain,job_spaces_redeveloped_orig,job_spaces_redeveloped_new,job_spaces_developed,resunits_undevelopable,resunits_remain,resunits_redeveloped_orig,resunits_redeveloped_new,resunits_developed,density_hhemp_orig,density_hhemp_new,density_hhemp_change
0,0,599.400903,1.220382e+04,4,a4,"{'rings': [[[420271.3705000002, 4480517.338500...",Bluffdale,2.0,Single Family A4,4,2031.0,2707.052025,114.354702,807.318475,123,65,-58,1363,1301,-62,8.981725,56.600356,57.967360,0.0,0.0,3.997729,1297.168923,61.862534,0.0,0.0,0.5,0.5,0.0
1,0,599.400903,1.220382e+04,4,a4,"{'rings': [[[420271.3705000002, 4480517.338500...",Bluffdale,4.0,Single Family A4,4,24.0,2.978770,0.000000,1.724645,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
2,1,1554.841774,8.561669e+04,4,a4,"{'rings': [[[420636.9475999996, 4481132.772299...",Bluffdale,2.0,Single Family A4,4,2031.0,2707.052025,114.354702,807.318475,123,65,-58,1363,1301,-62,8.981725,56.600356,57.967360,0.0,0.0,3.997729,1297.168923,61.862534,0.0,0.0,0.5,0.5,0.0
3,1,1554.841774,8.561669e+04,4,a4,"{'rings': [[[420636.9475999996, 4481132.772299...",Bluffdale,4.0,Single Family A4,4,24.0,2.978770,0.000000,1.724645,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
4,2,254.967270,3.811046e+03,4,a4,"{'rings': [[[420671.3481999999, 4481261.932600...",Bluffdale,2.0,Single Family A4,4,2031.0,2707.052025,114.354702,807.318475,123,65,-58,1363,1301,-62,8.981725,56.600356,57.967360,0.0,0.0,3.997729,1297.168923,61.862534,0.0,0.0,0.5,0.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12146,10970,10436.992442,3.292208e+06,10,h2,"{'rings': [[[412703.1934000002, 4553118.742900...",Clearfield,1.0,Mixed Residential Suburban,35,7331.0,1872.119633,189.304046,189.018404,1750,16843,15093,8450,10718,2268,865.662970,807.061567,77.677413,7591.0,7580.0,129.621644,7554.581411,766.497079,1518.0,1516.0,5.4,14.7,9.3
12147,10971,2229.618596,2.944859e+05,10,g3,"{'rings': [[[414942.4210000001, 4550112.225299...",Clearfield,1.0,Mixed-Use City/TOD,32,512.0,256.808450,58.686044,102.321081,2689,35798,33109,543,6248,5705,224.099551,1176.170012,1289.023703,12538.0,21860.0,5.024905,511.558476,27.140651,2089.0,3643.0,12.6,163.7,151.1
12148,10972,856.947230,2.329713e+04,10,g3,"{'rings': [[[417057.5292999996, 4550162.428400...",Clearfield,1.0,Mixed-Use City/TOD,32,512.0,256.808450,58.686044,102.321081,2689,35798,33109,543,6248,5705,224.099551,1176.170012,1289.023703,12538.0,21860.0,5.024905,511.558476,27.140651,2089.0,3643.0,12.6,163.7,151.1
12149,10973,966.598419,5.007495e+04,10,g3,"{'rings': [[[416772.02419999987, 4550546.96120...",Clearfield,1.0,Mixed-Use City/TOD,32,512.0,256.808450,58.686044,102.321081,2689,35798,33109,543,6248,5705,224.099551,1176.170012,1289.023703,12538.0,21860.0,5.024905,511.558476,27.140651,2089.0,3643.0,12.6,163.7,151.1


In [34]:
#create map centered on Salt Lake
map_parcels_areas_add = gis.map('Salt Lake')

#expression to classify employment growth
arcade_expression_jobspaceschange = ("var v = $feature.job_spaces_change;"
                                     "if      (v<0    ) { return 'class0'; }"
                                     "if      (v<2500 ) { return 'class1'; }"
                                     "else if (v<5000 ) { return 'class2'; }"
                                     "else if (v<10000) { return 'class3'; }"
                                     "else              { return 'class4'; }")

#symbology for enrollment classes
uv_jobspaceschange = [{"value":"class0", "label":"Job Decline"     , "symbol":{"type":"esriSFS","color":[  0,  0,  0,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
                      {"value":"class1", "label":"0 to 2,500"      , "symbol":{"type":"esriSFS","color":[130,165,217,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
                      {"value":"class2", "label":"2,500 to 5,000"  , "symbol":{"type":"esriSFS","color":[140,125,164,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
                      {"value":"class3", "label":"5,000 to 10,000" , "symbol":{"type":"esriSFS","color":[149,85,111,168],  "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
                      {"value":"class4", "label":"More than 10,000", "symbol":{"type":"esriSFS","color":[168,4,4,168],     "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}}]


#define sdf layer
sdf_AANewHHEmp.spatial.plot(map_widget = map_parcels_areas_add,
                            renderer_type='u-a', #'u-a' stands for uniqe value with arcade expression
                            unique_values=uv_jobspaceschange,
                            arcade_expression=arcade_expression_jobspaceschange,
                            default_symbol="" #don't include an 'other' category
                            )

#define map characteristics
map_parcels_areas_add.layout.height='500px'
map_parcels_areas_add.legend=True

#map title
display(Markdown('<h2><center>Job Growth by Analysis Area</center></h2>'))

#display map
map_parcels_areas_add

<h2><center>Job Growth by Analysis Area</center></h2>

MapView(layout=Layout(height='500px', width='100%'), legend=True)

In [35]:
#create map centered on Salt Lake
map_parcels_areas_addHH = gis.map('Salt Lake')

#expression to classify employment growth
arcade_expression_resunitschange = ("var v = $feature.resunits_change;"
                                    "if      (v<0   ) { return 'class0'; }"
                                    "if      (v<1000) { return 'class1'; }"
                                    "else if (v<2000) { return 'class2'; }"
                                    "else if (v<5000) { return 'class3'; }"
                                    "else             { return 'class4'; }")

#symbology for enrollment classes
uv_resunitschange = [{"value":"class0", "label":"HH Decline"     , "symbol":{"type":"esriSFS","color":[  0,  0,  0,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
                     {"value":"class1", "label":"0 to 1,000"     , "symbol":{"type":"esriSFS","color":[130,165,217,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
                     {"value":"class2", "label":"1,000 to 2,000" , "symbol":{"type":"esriSFS","color":[140,125,164,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
                     {"value":"class3", "label":"2,000 to 5,000" , "symbol":{"type":"esriSFS","color":[149, 85,111,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
                     {"value":"class4", "label":"More than 5,000", "symbol":{"type":"esriSFS","color":[168,  4,  4,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}}]


#define sdf layer
sdf_AANewHHEmp.spatial.plot(map_widget = map_parcels_areas_addHH,
                            renderer_type='u-a', #'u-a' stands for uniqe value with arcade expression
                            unique_values=uv_resunitschange,
                            arcade_expression=arcade_expression_resunitschange,
                            default_symbol="" #don't include an 'other' category
                            )

#define map characteristics
map_parcels_areas_addHH.layout.height='500px'
map_parcels_areas_addHH.legend=True

#map title
display(Markdown('<h2><center>Household Growth by Analysis Area</center></h2>'))

#display map
map_parcels_areas_addHH

<h2><center>Household Growth by Analysis Area</center></h2>

MapView(layout=Layout(height='500px', width='100%'), legend=True)

In [36]:
sdf_AA

,FID,city,county,city_lu_ty,gen_lu_typ,max_dua,code,plan_year,plan_sourc,data_sourc,Shape_Leng,Shape_Area,AnalysisAr,Analysis_1,SHAPE
0,0,Bluffdale,Salt Lake,Cluster Residential,Residential SF,1.0,a4,2019.0,,,599.400903,12203.821403,Bluffdale,a4,"{""rings"": [[[420271.3705000002, 4480517.338500..."
1,1,Bluffdale,Salt Lake,Cluster Residential,Residential SF,1.0,a5,2019.0,,,5673.401678,395913.547830,Bluffdale,a5,"{""rings"": [[[421136.0301000001, 4481104.8302],..."
2,2,Bluffdale,Salt Lake,Cluster Residential,Residential SF,1.0,a4,2019.0,,,1554.841774,85616.693206,Bluffdale,a4,"{""rings"": [[[420636.9475999996, 4481132.772299..."
3,3,Bluffdale,Salt Lake,Cluster Residential,Residential SF,1.0,a4,2019.0,,,254.967270,3811.045695,Bluffdale,a4,"{""rings"": [[[420671.3481999999, 4481261.932600..."
4,4,Bluffdale,Salt Lake,Cluster Residential,Residential SF,1.0,a5,2019.0,,,2588.398617,261893.287379,Bluffdale,a5,"{""rings"": [[[421931.24289999995, 4482241.10689..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11136,11136,Payson,Utah,Mixed Use Neighborhood,Mixed Use,12.0,g1,2020.0,https://paysonutah.org/storage/2016/05/General...,https://paysonutah.org/storage/2016/05/General...,101.746609,647.000097,Payson,g1,"{""rings"": [[[437436.2742999997, 4432295.444599..."
11137,11137,Payson,Utah,Mixed Use Neighborhood,Mixed Use,12.0,g1,2020.0,https://paysonutah.org/storage/2016/05/General...,https://paysonutah.org/storage/2016/05/General...,347.071602,6455.345939,Payson,g1,"{""rings"": [[[437517.15270000044, 4432410.1381]..."
11138,11138,Payson,Utah,Mixed Use Neighborhood,Mixed Use,12.0,g1,2020.0,https://paysonutah.org/storage/2016/05/General...,https://paysonutah.org/storage/2016/05/General...,441.004580,8873.413053,Payson,g1,"{""rings"": [[[437390.2474999996, 4432411.0495],..."
11139,11139,Payson,Utah,Mixed Use Neighborhood,Mixed Use,12.0,g1,2020.0,https://paysonutah.org/storage/2016/05/General...,https://paysonutah.org/storage/2016/05/General...,743.393043,29068.043955,Payson,g1,"{""rings"": [[[439725.0575000001, 4433219.461200..."
